In [22]:
!pip install PyPDF2
!pip install pymupdf Pillow pytesseract

In [84]:
from PyPDF2 import PdfReader
import fitz  # PyMuPDF
import os
import shutil

def get_pic(page_num):
    pdf_path = "QA.pdf"
    folder = "images"


    doc = fitz.open(pdf_path)
    
    image_list = doc.get_page_images(page_num)
    
    for img_index, img in enumerate(image_list):
        xref = img[0]
        pix = fitz.Pixmap(doc, xref)
        pix = fitz.Pixmap(fitz.csRGB, pix)
        pix.save(os.path.join(folder, "page{}_img{}.png".format(page_num-6, img_index)))

def read(file_path, start, end):
    
    result = []
    
    with open(file_path, 'rb') as file:
        pdf_reader = PdfReader(file)
        num_pages = len(pdf_reader.pages)
        if end > num_pages:
            print("end number outside of total pages")
            return None


        folder = "images"

        if os.path.exists(folder):
            shutil.rmtree(folder)
    
        os.makedirs(folder)
        q = 0
        
        for page_num in range(start, end):
            page = pdf_reader.pages[page_num]
            text = page.extract_text()

            if "Answers and Explanations" in text:
                print("bad")
                q = 0

            if "Multiple Choice Questions" in text:
                print("good")
                q = 1

            if q == 1:
                get_pic(page_num)
                q = 0
                
            result.append(text)
            print(text)

    return "\n".join(result)

In [86]:
textbook = "QA.pdf"
contents = read(textbook, 9, 191)
questions_d, answers_d = get_questions_and_answers(contents)
#answers_d = clean_sentences(answer)
#print(answers_d)

good
1
1Cellular Pathology, Inflammation, 
and Repair
 Multiple Choice Questions
 1. Programmed cell death is also called as:
 A. Degeneration
 B. Calcification
 C. Apoptosis
 D. Necrosis
 E. Atrophy
 2. Which of the following medical condition is because of 
inherited defect in DNA repair?
 A. Acanthosis nigricans
 B. Xeroderma pigmentosum
 C. Basal cell nevus syndrome
 D. Bloom syndrome
 E. Werner syndrome
 3. Which of the following is a pathologic cause of 
hyperplasia?
 A. Endometrial hyperplasia
 B. Compensatory hyperplasia after partial hepatectomy
 C. Hormonal stimulation seen in breast development at 
puberty
 D. Antigenic stimulation seen in lymphoid hyperplasia
 E. Cardiac muscle in hypertension
 4. In pathology, what is the name given when there is a 
reversible change of one cell type to another?
 A. Hyperplasia
 B. Hypertrophy
 C. Metaplasia
 D. Dysplasia
 E. Atrophy
 5. Which of the following is the best statement for 
lipofuscin?
 A. Perinuclear yellow-brown pigment
 B. 

In [24]:
import re

def filter(text):
    def license_filter(text):
        pattern = r"© The Author\(s\), under exclusive license to Springer Nature Switzerland AG \d{4}\s+V\s*\.\s*K\s*\.\s*Kohli\s*et\s*al\.,\s*Comprehensive Multiple-Choice Questions in Pathology\s*,\s*https://doi\.org/10\.1007/[\d\w\-_.]+"
        return re.sub(pattern, "", text)

    text = text.replace("\xa0", " ")
    text = license_filter(text)
    return text


In [25]:
import numpy as np 

def get_questions_and_answers(text):
    questions = []
    answers = []

    current_Q = []
    current_A = []

    q = True
    a = False
    filtered_text = filter(text)

    for line in filtered_text.split('\n')[4:]:
        line = line.strip()
        if q:
            if "Answers and Explanations" in line:
                questions.append(current_Q)
            
                current_Q = []
                q = False
                a = True
                continue

            
            current_Q.append(line)

        if a:
            if "Bibliography" in line:
                answers.append(current_A)
                current_A = []
                q = False
                a = False
                continue
                
            current_A.append(line)

        if q * a == 0:
            if "Multiple Choice Questions" in line:
                q = True
                a = False
            
    return questions, answers

In [26]:
def restructure_questions(questions):
    result = []
    
    def split_q(text):
        pattern = r"(?=\b\d{1,2}\.\s|A\.\s*|B\.\s*|C\.\s*|D\.\s*|E\.\s*)"
        split_text = re.split(pattern, text)
        return [part.strip() for part in split_text if part.strip() and len(part) > 1]

    y = 1
    
    for question_block in questions:
        text = " ".join(question_block)
        lists = split_q(text)
        current = []
        x = 1
        
        for i in lists:
            if re.match(r'^\d{1,2}\.\s', i):
                if current:
                    result.append([f"Set_{y}: q{x}"] + current)
                    x += 1
                current = [i[3:]]
            else: 
                current.append(i)
        
        if current:
            result.append([f"Set_{y}: q{x}"] + current)
        y += 1
    
    return result

In [27]:
def restructure_answers(answers):
    result = []
    def split_a(text):
        pattern = r'(?<!\d)\b\d+\.\s+(?=[A])'
        split_text = re.split(pattern, text)
        return np.array([part.strip() for part in split_text if part.strip()])
    y = 1
    for answer in answers:
        x = 1
        text = "\n".join(answer)
        lists = split_a(text)
            

        
        for line in lists:
           # line = re.sub(r'^Answer: ', '', line)
            current = [f"Set_{y}: q{x}"] + [line[:line.find("\n")], line[line.find("\n")+1:].replace("\n", " ")]
            result.append(current)
            x += 1
        y += 1
    return result
        

In [99]:
def remove_answers_and_explanations(text):
    pattern1 = r'Answers and Explanations.*?((?<!\.)\s|$)'
    cleaned_text = re.sub(pattern1, '', text, flags=re.DOTALL).strip()
    #pattern = r"(?<![A-E]\.)(?<=\.\s*)[^\s.]+(\s+[^\s.]+)*\s*$"
    #cleaned_text = re.sub(pattern, '.', text).strip()
    return cleaned_text

def clean_sentences(double_list):
    return [[remove_answers_and_explanations(sentence) for sentence in sublist] for sublist in double_list]



def ending(text):
    pattern = r"\.\s*[^\s.]+(\s+[^\s.]+)*\s*$"
    modified_text = re.sub(pattern, '.', text).strip()
    return modified_text

def explanation(answers):
    for answer in answers:
        if len(answer) >= 3:
            answer[2] = ending(answer[2])
    return answers


def modify_choice(text):
    pattern = r'Multiple Choice Questions.*'
    cleaned_text = re.sub(pattern, '', text, flags=re.DOTALL).strip()
    return cleaned_text
def clean_choice(double_list):
    return [[modify_choice(sentence) for sentence in sublist] for sublist in double_list]

In [101]:
textbook = "QA.pdf"
contents = read(textbook, 9, 191)
questions_d, answers_d = get_questions_and_answers(contents)
#answers_d = clean_sentences(answer)
#print(answers_d)

good
1
1Cellular Pathology, Inflammation, 
and Repair
 Multiple Choice Questions
 1. Programmed cell death is also called as:
 A. Degeneration
 B. Calcification
 C. Apoptosis
 D. Necrosis
 E. Atrophy
 2. Which of the following medical condition is because of 
inherited defect in DNA repair?
 A. Acanthosis nigricans
 B. Xeroderma pigmentosum
 C. Basal cell nevus syndrome
 D. Bloom syndrome
 E. Werner syndrome
 3. Which of the following is a pathologic cause of 
hyperplasia?
 A. Endometrial hyperplasia
 B. Compensatory hyperplasia after partial hepatectomy
 C. Hormonal stimulation seen in breast development at 
puberty
 D. Antigenic stimulation seen in lymphoid hyperplasia
 E. Cardiac muscle in hypertension
 4. In pathology, what is the name given when there is a 
reversible change of one cell type to another?
 A. Hyperplasia
 B. Hypertrophy
 C. Metaplasia
 D. Dysplasia
 E. Atrophy
 5. Which of the following is the best statement for 
lipofuscin?
 A. Perinuclear yellow-brown pigment
 B. 

In [105]:
questions = restructure_questions(questions_d)
questions = clean_choice(questions)

[['Set_1: q1', 'Programmed cell death is also called as:', 'A. Degeneration', 'B. Calcification', 'C. Apoptosis', 'D. Necrosis', 'E. Atrophy'], ['Set_1: q2', 'Which of the following medical condition is because of inherited defect in DNA repair?', 'A. Acanthosis nigricans', 'B. Xeroderma pigmentosum', 'C. Basal cell nevus syndrome', 'D. Bloom syndrome', 'E. Werner syndrome'], ['Set_1: q3', 'Which of the following is a pathologic cause of hyperplasia?', 'A. Endometrial hyperplasia', 'B. Compensatory hyperplasia after partial hepatectomy', 'C. Hormonal stimulation seen in breast development at puberty', 'D. Antigenic stimulation seen in lymphoid hyperplasia', 'E. Cardiac muscle in hypertension'], ['Set_1: q4', 'In pathology, what is the name given when there is a reversible change of one cell type to another?', 'A. Hyperplasia', 'B. Hypertrophy', 'C. Metaplasia', 'D. Dysplasia', 'E. Atrophy'], ['Set_1: q5', 'Which of the following is the best statement for lipofuscin?', 'A. Perinuclear y

In [31]:
answers = restructure_answers(answers_d)
answers = clean_sentences(answers)
answers = explanation(answers)
#print(answers)

In [32]:
import pandas as pd

q_table = pd.DataFrame(data=questions)
q_table.columns = ["id", "Question", "A", "B", "C", "D", "E", "F"]

a_table = pd.DataFrame(data=answers)
a_table.columns = ["id", "Answer", "explaination"]

In [33]:
a_table = a_table[a_table["Answer"].str[:6] == "Answer"].reset_index()

In [34]:
q_table.dropna(how='all')
q_table.to_csv('questions.csv', index=False)

In [35]:
a_table.dropna(how='all')
a_table.to_csv('answers.csv', index=False)

In [36]:
a_table_m = pd.read_csv('answers.csv')
a_table_m['Answer'] = a_table_m['Answer'].str.replace(r'^Answer:\s*', '', regex=True)
a_table_m.to_csv('answers.csv', index=False)

In [37]:
new_row = {
    'id': 'Set_22: q2', 
    'Answer': 'C. Meningioma', 
    'explaination': 'Meningiomas are relatively common neoplasms derived from meningothelial cells of the arachnoid. The World Health organization (WHO) classified meningiomas in three groups'
}


row = pd.DataFrame([new_row])
#print(a_table_m.head())
#print(row)

a_table_m = pd.concat([a_table_m.iloc[:572],row, a_table_m.iloc[572:]]).reset_index(drop=True)
a_table_m.to_csv('answers.csv', index=False) ##这里跑一次就行

In [108]:
q_table_f = pd.read_csv('questions_f.csv')
print(q_table_f)
a_table_f = pd.read_csv('answers.csv')

              id                                           Question  \
0      Set_1: q1           Programmed cell death is also called as:   
1      Set_1: q2  Which of the following medical condition is be...   
2      Set_1: q3  Which of the following is a pathologic cause o...   
3      Set_1: q4  In pathology, what is the name given when ther...   
4      Set_1: q5  Which of the following is the best statement f...   
..           ...                                                ...   
623  Set_23: q19   Which of the following eye conditions lead to...   
624  Set_23: q20   Which of the following eye conditions have pl...   
625  Set_23: q21   A 61-year-old female presents with painless p...   
626  Set_23: q22   A 42-year-old female presents for an evaluati...   
627  Set_23: q23   A 51-year-old male presents with bilateral bl...   

                                       A  \
0                        A. Degeneration   
1                A. Acanthosis nigricans   
2             A

In [39]:
table = pd.concat([q_table_f.drop(["id"], axis = 1), a_table_f.drop(["id"], axis = 1)], axis=1)
table.to_csv('final.csv', index=False)

In [40]:
a_table_f.drop(columns=['index'])
table_json = pd.concat([q_table_f.drop(["id"], axis = 1), a_table_f.drop(["id"], axis = 1)], axis=1)
table_json.to_csv('final.csv', index=False)

image test below

In [42]:
pip install pymupdf

Note: you may need to restart the kernel to use updated packages.


In [43]:
"""
import fitz  # PyMuPDF
import os
import shutil

# Path to the PDF file
pdf_path = "QA.pdf"
# Path to the folder where images will be saved
folder = "images"

if os.path.exists(folder):
    shutil.rmtree(folder)
    
os.makedirs(folder)

doc = fitz.open(pdf_path)

for i in range(10,190):
    # Get the list of images on the page
    image_list = doc.get_page_images(i)
    
    for img_index, img in enumerate(image_list):
        xref = img[0]
        pix = fitz.Pixmap(doc, xref)
        pix = fitz.Pixmap(fitz.csRGB, pix)
        pix.save(os.path.join(folder, "page{}_img{}.png".format(i-8, img_index)))

print("Image extraction completed.")
"""





'\nimport fitz  # PyMuPDF\nimport os\nimport shutil\n\n# Path to the PDF file\npdf_path = "QA.pdf"\n# Path to the folder where images will be saved\nfolder = "images"\n\nif os.path.exists(folder):\n    shutil.rmtree(folder)\n    \nos.makedirs(folder)\n\ndoc = fitz.open(pdf_path)\n\nfor i in range(10,190):\n    # Get the list of images on the page\n    image_list = doc.get_page_images(i)\n    \n    for img_index, img in enumerate(image_list):\n        xref = img[0]\n        pix = fitz.Pixmap(doc, xref)\n        pix = fitz.Pixmap(fitz.csRGB, pix)\n        pix.save(os.path.join(folder, "page{}_img{}.png".format(i-8, img_index)))\n\nprint("Image extraction completed.")\n'